In [ ]:
import json
import glob
import itertools

import numpy as np
import pandas as pd

from qiskit import transpile, QuantumCircuit
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator, BackendEstimator
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp


import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import (
    GCNConv, 
    TransformerConv, 
    GATv2Conv, 
    global_mean_pool, 
    Linear, 
    ChebConv, 
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from pprint import pprint

from qiskit.opflow import I, X, Z
from qiskit.primitives import Estimator
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.minimum_eigensolvers import VQE, VQEResult
from qiskit.algorithms.optimizers import SLSQP, SPSA, COBYLA

from qiskit_aer.primitives import Estimator as AerEstimator

In [ ]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)
ideal_estimator = AerEstimator() #Estimator()
aer_esimator = AerEstimator()
backend_estimator = BackendEstimator(backend)

In [ ]:
operator = (-1.052373245772859 * I ^ I) + \
            (0.39793742484318045 * I ^ Z) + \
            (-0.39793742484318045 * Z ^ I) + \
            (-0.01128010425623538 * Z ^ Z) + \
            (0.18093119978423156 * X ^ X)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
ansatz.num_qubits = operator.num_qubits

In [ ]:
transpiled_ansatz = transpile(ansatz, backend, optimization_level=0)

In [ ]:
### Create training dataset

In [ ]:
N_ENTRIES = 500

obs = encode_pauli_sum_op(operator)

entries = []
for i in tqdm_notebook(range(N_ENTRIES)):
    bounded_circuit = ansatz.bind_parameters(np.random.uniform(-2, 2, (8)))
    circuit = transpile(bounded_circuit, backend, optimization_level=0)

    exp_vals = create_meas_data_from_estimators(
        circuits=bounded_circuit, 
        observables=operator, 
        estimators=[ideal_estimator, backend_estimator],
        shots=10000
    )
    ideal_exp_val, noisy_exp_val = exp_vals
    
    graph_data = circuit_to_graph_data_json(
        circuit=circuit,
        properties=properties,
        use_qubit_features=True,
        use_gate_features=True,
    )

    entry = ExpValueEntry(
        circuit_graph=graph_data,
        observable=obs,
        ideal_exp_value=ideal_exp_val,
        noisy_exp_values=[
            noisy_exp_val
        ],
        circuit_depth=circuit.depth()
    )
    entries.append(entry.to_dict())
    
    if i > 0 and i % 100 == 0:
        with open(f"./data/demo_vqe_dataset/vqe_dataset_{i}.json", "w") as json_file:
            json.dump(entries, json_file)
        entries = []
        
with open(f"./data/demo_vqe_dataset/vqe_dataset_{i}.json", "w") as json_file:
    json.dump(entries, json_file)

In [ ]:
### load data
import random

In [ ]:
paths = glob.glob("./data/demo_vqe_dataset/vqe_dataset_**.json")
# random.shuffle(paths)
train_paths = paths[:3]
val_paths = paths[3:]

In [ ]:
BATCH_SIZE = 32

train_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        train_paths,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        val_paths,
    ),
    batch_size=BATCH_SIZE,
    shuffle=False
)

for data in train_loader:
    print(data)
    break

In [ ]:
### Model

In [ ]:
class ExpValCircuitGraphModel(torch.nn.Module):
    def __init__(
        self,
        n_qubits: int,
        num_node_features: int,
        hidden_channels: int
    ):
        super().__init__()
        
        self.transformer1 = TransformerConv(
            num_node_features, hidden_channels,
            heads=3,
            dropout=0.1
        )
        self.pooling1 = ASAPooling(hidden_channels * 3, 0.5)

        self.transformer2 = TransformerConv(
            hidden_channels * 3, hidden_channels,
            heads=2,
            dropout=0.1
        )
        self.pooling2 = ASAPooling(hidden_channels * 2, 0.5)
        
        self.body_seq = torch.nn.Sequential(
            Linear(hidden_channels * 2 + 2, hidden_channels),
            torch.nn.Dropout(0.2),
            Linear(hidden_channels, 1)
        )

    def forward(self, 
                exp_value, observable, 
                circuit_depth, nodes, 
                edge_index, batch):
        graph = self.transformer1(nodes, edge_index)    
        graph, edge_index, _, batch, _ = self.pooling1(
            graph, edge_index, batch=batch
        )
        
        graph = self.transformer2(graph, edge_index)
        graph, edge_index, _, batch, _ = self.pooling2(
            graph, edge_index, batch=batch
        )
        
        graph = global_mean_pool(graph, batch)
    
        merge = torch.cat((
            graph, 
            exp_value,
            circuit_depth
        ), dim=1)

        return self.body_seq(merge)

In [ ]:
model = ExpValCircuitGraphModel(
    n_qubits=5,
    num_node_features=28, 
    hidden_channels=10
)
criterion = torch.nn.MSELoss()

In [ ]:
optimizer = Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, 
                              'min', 
                              factor=0.1, 
                              patience=15, 
                              verbose=True, 
                              min_lr=0.00001)

In [ ]:
min_valid_loss = np.inf

train_losses = []
val_losses = []

N_EPOCHS = 30

progress = tqdm_notebook(range(N_EPOCHS), desc='Model training', leave=True)
for epoch in progress:
    train_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        out = model(
            data.noisy_0, 
            data.observable, 
            data.circuit_depth,
            data.x, 
            data.edge_index, 
            data.batch
        )
        loss = criterion(out, data.y)        
        
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
                
    valid_loss = 0.0
    model.eval()
    for i, data in enumerate(val_loader):
        out = model(
            data.noisy_0, 
            data.observable, 
            data.circuit_depth, 
            data.x, 
            data.edge_index, 
            data.batch)
        loss = criterion(out, data.y)        

        valid_loss += loss.item()

    scheduler.step(valid_loss)

    if epoch >= 1:    
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(valid_loss / len(val_loader))

        progress.set_description(f"{round(train_losses[-1], 5)}, {round(val_losses[-1], 5)}")
        progress.refresh()

In [ ]:
plt.plot(train_losses, label="train_loss")
plt.plot(val_losses, label="val_loss")
plt.yscale('log')

plt.legend()
plt.show()

distances = []
for i, data in enumerate(val_loader):
    out = model(data.noisy_0, data.observable, data.circuit_depth, data.x, data.edge_index, data.batch)
    
    for ideal, noisy, ngm_mitigated in zip(
        data.y.tolist(),
        data.noisy_0.tolist(),
        out.tolist()
    ):
        ideal = np.mean(ideal)
        noisy = np.mean(noisy)
        ngm_mitigated = np.mean(ngm_mitigated)
        distances.append({
            "ideal": ideal,
            "noisy": noisy,
            "ngm_mitigated": ngm_mitigated,
            "dist_noisy": np.abs(ideal - noisy),
            "dist_ngm": np.abs(ideal - ngm_mitigated),
        })
        
df = pd.DataFrame(distances)
sns.boxplot(data=df[["dist_noisy", "dist_ngm"]], orient="h", showfliers = False)
plt.title("Dist to ideal exp value")
plt.show()

sns.histplot([df['ideal'], df['noisy'], df["ngm_mitigated"]], kde=True, bins=40)
plt.title("Exp values distribution")
plt.show()

In [ ]:
ideal_vals = []
ideal_callback = lambda count, params, mean, std: ideal_vals.append(mean)

noisy_vals = []
noisy_callback = lambda count, params, mean, std: noisy_vals.append(mean)

mitigated_vals = []
mitigated_callback = lambda count, params, mean, std: mitigated_vals.append(mean)

default_callback = lambda count, params, mean, std: print(params)

initial_point =  np.ones(8) #np.random.random(8)

NgemEstimator = ngem(BackendEstimator, model=model, backend=backend)
ngem_estimator = NgemEstimator(backend, skip_transpilation=True)

for estimator, callback in zip(
    [ideal_estimator, ngem_estimator, backend_estimator],
    [ideal_callback, mitigated_callback, noisy_callback]
):
    print(f"{estimator} VQE execution...")
    vqe_optimizer = SPSA(maxiter=100)
    vqe = VQE(estimator, ansatz, vqe_optimizer, callback=callback, initial_point=initial_point)
    result = vqe.compute_minimum_eigenvalue(operator)

In [ ]:
plt.plot(ideal_vals, label="ideal")
plt.plot(mitigated_vals, label="mitigated")
plt.plot(noisy_vals, label="noisy")
plt.legend()
# plt.yscale("log")
plt.show()